In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/ZetaGroup1/WeatherMachine/refs/heads/main/Final_data_scripts/Data/predicted_snow_conditions.csv')

df

,Date,cloud_cover_code,Precipitation amount [mm],Snow depth [cm],Average temperature [°C],Direct solar radiation mean [W/m2],days_above_10cm,days_above_20cm,Predicted Snow Depth [cm]
0,2005-01-02,3.0,4.2,15.0,0.2,1.1,1,0,13.891783
1,2005-01-03,3.0,0.8,19.0,-1.4,0.7,2,0,15.793369
2,2005-01-04,3.0,1.8,19.0,-1.8,1.8,3,0,17.248934
3,2005-01-05,3.0,6.2,20.0,-1.5,1.8,4,0,17.672102
4,2005-01-06,3.0,5.0,27.0,-1.3,1.7,5,1,17.215079
...,...,...,...,...,...,...,...,...,...
29221,2024-12-28,1.0,-1.0,31.0,-2.9,1.5,33,13,18.218463
29222,2024-12-29,8.0,1.9,31.0,-0.7,0.5,34,14,19.594800
29223,2024-12-30,3.0,0.8,32.0,-5.3,42.0,35,15,25.966975
29224,2024-12-31,8.0,-1.0,34.0,-2.2,61.7,36,16,23.966696


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29226 entries, 0 to 29225
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Date                                29226 non-null  object 
 1   cloud_cover_code                    29226 non-null  float64
 2   Precipitation amount [mm]           29226 non-null  float64
 3   Snow depth [cm]                     29226 non-null  float64
 4   Average temperature [°C]            29226 non-null  float64
 5   Direct solar radiation mean [W/m2]  29226 non-null  float64
 6   days_above_10cm                     29226 non-null  int64  
 7   days_above_20cm                     29226 non-null  int64  
 8   Predicted Snow Depth [cm]           29226 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 2.0+ MB


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# Function to check if there's a continuous period of n days with snow depth above a certain threshold
def check_continuous_snow(df, threshold, days_required):
    # Filter data with snow depth >= threshold
    snow_days = df[df['Snow depth [cm]'] >= threshold]
    
    # Check if there's a continuous period of 'days_required' days
    snow_days['date_diff'] = snow_days['Date'].diff().dt.days.fillna(1)
    snow_days['group'] = (snow_days['date_diff'] > 1).cumsum()

    # Count the number of consecutive days for each group
    consecutive_days = snow_days.groupby('group').size()
    
    # Return True if any group has at least 'days_required' consecutive days
    return any(consecutive_days >= days_required)

# Function to check snow conditions for each year
def check_snow_conditions(df, year):
    # Filter the dataframe for the specific year
    df_year = df[df['Date'].dt.year == year]
    
    # Check for continuous 14 days of snow with at least 20 cm
    result_14_days_20cm = check_continuous_snow(df_year, 20, 14)
    
    # Check for continuous 28 days of snow with at least 10 cm
    result_28_days_10cm = check_continuous_snow(df_year, 10, 28)
    
    return result_14_days_20cm, result_28_days_10cm

# Loop through the years of interest
for year in range(2005, 2025):
    result_14, result_28 = check_snow_conditions(df, year)
    print(f"Year {year}:")
    print(f"  Continuous 14 days of snow with at least 20 cm: {result_14}")
    print(f"  Continuous 28 days of snow with at least 10 cm: {result_28}")

Year 2005:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2006:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2007:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2008:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2009:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2010:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2011:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2012:
  Continuous 14 days of snow with at least 20 cm: True
  Continuous 28 days of snow with at least 10 cm: True
Year 2013:
  Continuous 14 days 

C:\Users\Ilyas Land\AppData\Local\Temp\ipykernel_3300\2178498011.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snow_days['date_diff'] = snow_days['Date'].diff().dt.days.fillna(1)
C:\Users\Ilyas Land\AppData\Local\Temp\ipykernel_3300\2178498011.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snow_days['group'] = (snow_days['date_diff'] > 1).cumsum()
C:\Users\Ilyas Land\AppData\Local\Temp\ipykernel_3300\2178498011.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [6]:
df.to_csv('super.csv')

In [8]:
df["Date"] = pd.to_datetime(df["Date"])

df["below_20cm"] = df["Snow depth [cm]"] < 20

df["group"] = (df["below_20cm"] != df["below_20cm"].shift()).cumsum()

periods = df.groupby("group").agg({"Date": ["min", "max"], "below_20cm": "first", "Snow depth [cm]": "count"})
long_periods = periods[(periods["below_20cm", "first"] == True) & (periods["Snow depth [cm]", "count"] >= 14)]


for date_in, date_out in zip(long_periods["Date", "min"], long_periods["Date", "max"]):
    print(f"No at least 20 cm of snow from {date_in.date()} to {date_out.date()}")

No at least 20 cm of snow from 2005-03-26 to 2005-12-15
No at least 20 cm of snow from 2006-04-16 to 2006-10-31
No at least 20 cm of snow from 2006-11-16 to 2007-02-02
No at least 20 cm of snow from 2007-03-09 to 2008-01-30
No at least 20 cm of snow from 2008-02-08 to 2008-03-03
No at least 20 cm of snow from 2008-03-10 to 2008-03-23
No at least 20 cm of snow from 2008-04-02 to 2008-11-23
No at least 20 cm of snow from 2008-11-28 to 2009-01-22
No at least 20 cm of snow from 2009-01-24 to 2009-02-06
No at least 20 cm of snow from 2009-04-03 to 2009-12-26
No at least 20 cm of snow from 2010-04-07 to 2010-12-06
No at least 20 cm of snow from 2011-04-13 to 2012-01-12
No at least 20 cm of snow from 2012-04-16 to 2012-12-07
No at least 20 cm of snow from 2013-04-19 to 2015-01-13
No at least 20 cm of snow from 2015-03-09 to 2016-02-20
No at least 20 cm of snow from 2016-03-17 to 2017-02-22
No at least 20 cm of snow from 2017-03-17 to 2017-12-15
No at least 20 cm of snow from 2018-04-14 to 201